In [31]:
%pwd

'd:\\ML_project_with_MLFlow\\Complete-ML-Project-with-ML-Flow'

In [32]:
import os
os.chdir("d:\\ML_project_with_MLFlow\\Complete-ML-Project-with-ML-Flow")

In [33]:
%pwd

'd:\\ML_project_with_MLFlow\\Complete-ML-Project-with-ML-Flow'

In [34]:
from dataclasses import dataclass 
from pathlib import Path

In [35]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: Path
    local_data_file: Path
    unzip_dir : Path

In [49]:
from src.mlproject.constants import *
from box import ConfigBox

In [50]:
from src.mlproject.utils.common import read_yaml, create_directories

In [74]:
from pathlib import Path
class ConfigurationManager:
    def __init__(
            self,
            # config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(Path("config/config.yaml"))
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_configuration(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [69]:
import os
import urllib.request as request
import zipfile
from src.mlproject import logger
from src.mlproject.utils.common import get_size


In [70]:
class DataIngestion:
    def __init__(self,config : DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with the following info : \n {headers}")
        else:

            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [60]:
%pwd

'd:\\ML_project_with_MLFlow\\Complete-ML-Project-with-ML-Flow'

In [76]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_configuration()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2024-03-12 12:50:51,559: INFO: common :yaml_file : config\config.yaml loaded successfully]
[2024-03-12 12:50:51,820: INFO: common :yaml_file : params.yaml loaded successfully]
[2024-03-12 12:50:51,832: INFO: common :yaml_file : schema.yaml loaded successfully]
[2024-03-12 12:50:51,871: INFO: common :created directory at artifacts]
[2024-03-12 12:50:51,878: INFO: common :created directory at artifacts/data_ingestion]
[2024-03-12 12:50:55,964: INFO: 1153073202 :artifacts/data_ingestion/data.zip downloaded! with the following info : 
 Connection: close
Content-Length: 26148
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "744a98f53582af6c1e345731e31fea7ee6d50107320b580cda135b0798c6622a"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D1EE:1EA77C:291AE3:3843E7:65F00256
Accept-Ranges: bytes
Date

In [78]:
from ensure import ensure_annotations